# general vs. donations
1. 총 기부금액 ('funds_raised' vs. 'amount')
2. 총 기부건수 ('total_donations vs. 'chid'별 개수 count)

## 기본전제
- 기부금액이 0인 id: 1721, 1783, 1893, 2744는 donations엔 데이터가 없다 (당연히)
- 현재 general 기준 캠페인의 총 개수: 3904
- join한 테이블을 볼건데, 컬럼에 대한 설명은 아래와 같다.
    - funds_raised: general테이블의 총 기부금액
    - totlal_donations: general테이블의 총 기부건수
    - amount: donations테이블의 총 기부금액
    - currency: donations테이블 총 기부건수

In [2]:
import pandas as pd

In [3]:
general = pd.read_csv('general_dollar.csv')
donations = pd.read_csv('donations_dollar.csv')

In [20]:
# general과 donation 테이블 합치기
comp1 = general[['chid','title','created_date','funds_raised','total_donations']]
comp2 = donations.groupby('chid').agg({'amount':'sum', 'currency':"count"})
comp3 = comp1.merge(comp2, on='chid', how='left')
comp3

,chid,title,created_date,funds_raised,total_donations,amount,currency
0,1,Chris Beaty Memorial Fund,2020-06-03,175083.0,1705.0,119247.0,1089.0
1,2,1 percent chance,2020-06-21,299042.0,602.0,299042.0,602.0
2,3,MELS needs #100ComputersIn100Days,2020-06-19,13845.0,125.0,13845.0,125.0
3,4,100% to Victims of Taal Volcano in Philippines,2020-01-16,25.0,1.0,25.0,1.0
4,5,$100k for Center for Family Life,2020-11-02,13950.0,17.0,13950.0,17.0
...,...,...,...,...,...,...,...
3899,3901,Helping the Hansens: A Family in Crisis,2020-07-24,68691.0,673.0,68691.0,673.0
3900,3902,Zurn Strong,2020-10-13,41330.0,356.0,41330.0,356.0
3901,3903,Sean Culver recovery and rehab fund,2020-08-28,59205.0,562.0,59205.0,562.0
3902,3904,In loving memory of Koorosh (Cyrus) Yaghoubian,2020-08-09,58767.0,312.0,58767.0,312.0


### 1. general과 donations 내용 동일
- 총 개수: 3081
- 그렇다면, 금액과 건수 다른 것들은 3904 - 3081 = `823개`

In [21]:
# 두 테이블의 금액과 건수가 모두 같은 것만 가져오기
comp = comp3.assign(amount_diff = comp3['funds_raised'] != comp3['amount'],
                   counts_diff = comp3['total_donations'] != comp3['currency'])
same = comp[(comp['amount_diff'] == False) & (comp['counts_diff'] == False)]
same


,chid,funds_raised,total_donations,amount,currency,amount_diff,counts_diff
1,2,299042.0,602.0,299042.0,602.0,False,False
2,3,13845.0,125.0,13845.0,125.0,False,False
3,4,25.0,1.0,25.0,1.0,False,False
4,5,13950.0,17.0,13950.0,17.0,False,False
5,6,78380.0,252.0,78380.0,252.0,False,False
...,...,...,...,...,...,...,...
3899,3901,68691.0,673.0,68691.0,673.0,False,False
3900,3902,41330.0,356.0,41330.0,356.0,False,False
3901,3903,59205.0,562.0,59205.0,562.0,False,False
3902,3904,58767.0,312.0,58767.0,312.0,False,False


### 2. general과 donations 금액과 건수 둘 다 다른 데이터들
- 총 개수: 760
- 그렇다면, 금액과 건수 중 하나만 맞고, 하나는 틀린 것들은 823 - 760 = `63개`

In [25]:
# 두 테이블의 금액과 건수 둘 다 다른 것만 가져오기
comp = comp3.assign(amount_diff = comp3['funds_raised'] != comp3['amount'],
                   counts_diff = comp3['total_donations'] != comp3['currency'])
both_diff = comp[(comp['amount_diff'] == True) & (comp['counts_diff'] == True)]
both_diff = both_diff.rename(index=str, columns={"currency": "counts"})
both_diff.sort_values("funds_raised", ascending=False)



,chid,title,created_date,funds_raised,total_donations,amount,counts,amount_diff,counts_diff
3852,3854,Navajo & Hopi Families COVID-19 Relief Fund,2020-03-15,6074490.0,103620.0,74301.0,1089.0,True,True
3127,3129,Save Nil's Life!,2020-03-10,2402960.0,14655.0,148231.0,1089.0,True,True
2215,2217,Justice for Jacob Blake,2020-08-24,2296400.0,72914.0,62426.0,1089.0,True,True
2712,2714,Official Navajo Nation COVID-19 Relief Fund,2020-05-04,1831520.0,22040.0,126534.0,1089.0,True,True
2555,2557,Mercan'a can - Life for Mercan (SMA Tip 1-Type 1),2020-08-03,1536500.0,10193.0,694443.0,1089.0,True,True
...,...,...,...,...,...,...,...,...,...
3747,3749,VETERANS DIALYSIS CERTIFICATION,2020-02-11,1884.0,NaN,1873.0,22.0,True,True
1891,1893,I'm bankrupt due to unpaid wages.,2019-12-24,0.0,0.0,NaN,NaN,True,True
2742,2744,Overstayed my visa due to COVID19,2020-06-05,0.0,0.0,NaN,NaN,True,True
1719,1721,Help return to Canada,2020-04-06,0.0,0.0,NaN,NaN,True,True


### 3. general과 donations 중 금액만 동일하고 건수는 다른 데이터들
- 총 개수: 59개
- 그렇다면, 금액은 다르고 건수만 동일한 것들은 63 - 59 = `4개`

In [27]:
# 두 테이블의 금액만 같고, 건수는 다른 데이터 가져오기
comp = comp3.assign(amount_diff = comp3['funds_raised'] != comp3['amount'],
                   counts_diff = comp3['total_donations'] != comp3['currency'])
counts_diff = comp[(comp['amount_diff'] == False) & (comp['counts_diff'] == True)]
counts_diff

,chid,funds_raised,total_donations,amount,currency,amount_diff,counts_diff
20,21,63485.0,NaN,63485.0,652.0,False,True
22,23,68280.0,NaN,68280.0,402.0,False,True
318,319,15870.0,NaN,15870.0,54.0,False,True
329,330,50.0,NaN,50.0,1.0,False,True
429,430,55255.0,NaN,55255.0,148.0,False,True
437,438,52775.0,NaN,52775.0,337.0,False,True
538,539,51510.0,NaN,51510.0,943.0,False,True
545,546,97010.0,NaN,97010.0,873.0,False,True
571,572,57330.0,NaN,57330.0,335.0,False,True
660,661,57500.0,NaN,57500.0,59.0,False,True


### 4. general과 donations 중 금액은 다르고 건수만 동일한 데이터들
- 총 개수: 4개
- `일단, 계산은 맞은 것으로 보임`

In [13]:
# 두 테이블의 금액과 건수가 같은 것만 가져오기
comp = comp3.assign(amount_diff = comp3['funds_raised'] != comp3['amount'],
                   counts_diff = comp3['total_donations'] != comp3['currency'])
amount_diff = comp[(comp['amount_diff'] == True) & (comp['counts_diff'] == False)]
amount_diff

,chid,funds_raised,total_donations,amount,currency,amount_diff,counts_diff
242,243,47400.0,428.0,47475.0,428.0,True,False
1071,1073,57810.0,733.0,57720.0,733.0,True,False
2332,2334,81573.0,965.0,81578.0,965.0,True,False
2755,2757,5123220.0,533.0,5123221.0,533.0,True,False


## 특이사항 1
- 금액과 건수 모두 다른 데이터관련 (2번)
- 값을 보면 donations의 총 기부건수의 값 중 `1089가 유난히 겹친다.` 
    - 이를 바탕으로 추론한 결과, donations에서 스크래핑 할 수 있었던 기부자수의 MAX값이 1089였을 가능성 (그 이상의 건수가 없다)
    - 그래서 기부금액과 기부자수가 general과 donations에서 모두 다르게 나타난 것으로 파악된다.
    - 그러면, donations의 기부건수가 모두 1089건인것만 다르냐하면 그건 아니다.
    - 파악한 결과, 1089: `748개` / 1089 이하: `8개` / 전제조건에 말한 아예 기부자수가 없는 것: `4개` = `총합 760개`이다.

In [25]:
# 금액과 건수 모두 다른 데이터 중에 donations의 기부건수가 1089건인 것들
comp = comp3.assign(amount_diff = comp3['funds_raised'] != comp3['amount'],
                   counts_diff = comp3['total_donations'] != comp3['currency'])
both_diff = comp[(comp['amount_diff'] == True) & (comp['counts_diff'] == True)]
both_diff[both_diff['currency'] >= 1089]

,chid,funds_raised,total_donations,amount,currency,amount_diff,counts_diff
0,1,175083.0,1705.0,119247.0,1089.0,True,True
8,9,82165.0,1222.0,71965.0,1089.0,True,True
10,11,109564.0,1245.0,88337.0,1089.0,True,True
12,13,139808.0,2596.0,65857.0,1089.0,True,True
17,18,41080.0,1094.0,40920.0,1089.0,True,True
...,...,...,...,...,...,...,...
3883,3885,166120.0,7077.0,59828.0,1089.0,True,True
3884,3886,73307.0,2680.0,30455.0,1089.0,True,True
3887,3889,59412.0,1786.0,36720.0,1089.0,True,True
3894,3896,68554.0,3228.0,21228.0,1089.0,True,True


In [26]:
# 금액과 건수 모두 다른 데이터 중에 donations의 기부건수가 1089건 이하인 것들
comp = comp3.assign(amount_diff = comp3['funds_raised'] != comp3['amount'],
                   counts_diff = comp3['total_donations'] != comp3['currency'])
both_diff = comp[(comp['amount_diff'] == True) & (comp['counts_diff'] == True)]
both_diff[both_diff['currency'] <= 1088]

,chid,funds_raised,total_donations,amount,currency,amount_diff,counts_diff
587,588,82085.0,461.0,82062.0,460.0,True,True
839,840,51062.0,893.0,51012.0,892.0,True,True
880,881,166656.0,978.0,166644.0,977.0,True,True
1178,1180,67538.0,NaN,67408.0,577.0,True,True
1561,1563,2556.0,28.0,2551.0,27.0,True,True
1901,1903,58452.0,1087.0,58402.0,1086.0,True,True
3031,3033,44048.0,300.0,44023.0,299.0,True,True
3747,3749,1884.0,NaN,1873.0,22.0,True,True


## 특이사항 2
- 금액은 동일한데 건수만 다른 데이터 관련 (3번)
- 값을 보면 general의 'total_donations'가 `Null`값이 대부분인 것을 확인할 수 있다.
    - 금액은 동일하므로, general의 Null값을 donations의 기부건수로 채워주면 살릴 수 있는 데이터가 된다!
    - 그러면, general의 기부건수가 전부 Null이냐? 그건 아니다.
    - 파악한 결과, NaN: `58개` / 숫자가 다름: `1개` = `총합 59개`이다.
    - 58개는 살릴 수 있지 않을까?

In [31]:
# 금액은 동일한데 건수만 다른 것중 general의 기부건수가 Null값인 데이터 구하기
comp = comp3.assign(amount_diff = comp3['funds_raised'] != comp3['amount'],
                   counts_diff = comp3['total_donations'] != comp3['currency'])
counts_diff = comp[(comp['amount_diff'] == False) & (comp['counts_diff'] == True)]
counts_diff['total_donations'].isnull().sum()
#counts_diff[counts_diff['total_donations'].isnull()]

58

In [35]:
# 금액은 동일한데 건수만 다른 것중 general의 기부건수가 Null값이 아닌 데이터 구하기
comp = comp3.assign(amount_diff = comp3['funds_raised'] != comp3['amount'],
                   counts_diff = comp3['total_donations'] != comp3['currency'])
counts_diff = comp[(comp['amount_diff'] == False) & (comp['counts_diff'] == True)]
#counts_diff['total_donations'].notnull().sum()
counts_diff[counts_diff['total_donations'].notnull()]

,chid,funds_raised,total_donations,amount,currency,amount_diff,counts_diff
911,912,26201.0,19.0,26201.0,20.0,False,True


In [ ]:
df = general.merge(donations, on='chid', how='left')
df1 = df.assign(is_successful_g = general['funds_raised'] >= general['funds_target'])
df2 = df1.assign(is_successful_d = donation['funds_raised'] >= general['funds_target'])

In [21]:
comp = comp3.assign(amount_diff = comp3['funds_raised'] != comp3['amount'],
                   counts_diff = comp3['total_donations'] != comp3['currency'])
both_diff = comp[(comp['amount_diff'] == True) & (comp['counts_diff'] == True)]
both_diff[(both_diff['chid'] == 3896)]

,chid,scrape_date,created_date,funds_raised,total_donations,amount,currency,amount_diff,counts_diff
3894,3896,2020-11-03 18:45:00-05:00,2020-08-24,68554.0,3228.0,21228.0,1089.0,True,True


In [22]:
y = donations[['chid','created_at']]
y[(y['chid'] == 3896)].min()

chid                               3896
created_at    2020-10-27 00:12:00+00:00
dtype: object

In [23]:
y[(y['chid'] == 3896)].max()

chid                               3896
created_at    2020-11-03 22:53:06+00:00
dtype: object

In [ ]:
donations = pd.read_csv('donations_dollar.csv')

In [ ]:
comp = comp3.assign(amount_diff = comp3['funds_raised'] != comp3['amount'],
                   counts_diff = comp3['total_donations'] != comp3['currency'])
both_diff = comp[(comp['amount_diff'] == True) & (comp['counts_diff'] == True)]
both_diff[both_diff['currency'] <= 1088]